See https://github.com/ThomasWaldmann/py-esp32-ulp/ for the assembler for the ULP code

First section is the code compilation (binary is saved).  Then it can be used without 
loading up the assembler

In [8]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:5008
ho 0 tail 12 room 4
load:0x40078000,len:10600
ho 0 tail 12 room 4
load:0x40080400,len:5684
entry 0x400806bc
MicroPython v1.13 on 2020-09-02; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [ ]:

%sendtofile --mkdir --source /home/julian/extrepositories/py-esp32-ulp/esp32_ulp  esp32_ulp

In [81]:
from esp32_ulp.__main__ import src_to_binary

source = """\
dataiorec:  .long 0x00000000
timepin27:  .long 0x00000000
countpin27: .long 0x00000000
timepin12:  .long 0x00000000
countpin12: .long 0x00000000
entry:
            move r3, dataiorec
            move r1, 0x0000     // pin27 counter
            move r2, 0x0000     // pin12 counter
mainloop:
            reg_rd 0x3ff48424, 16, 17   // read IO register into r0
            st   r0, r3, 0      // dataiorec[0]=r0
            rsh  r0, r0, 14     // access pin 27
            and  r0, r0, 0x0001

            jump p27eq0, eq
            add  r1, r1, 0x0001
            jump p27eq0end
p27eq0:
            add  r1, r1, 0x0000 // update the zero flag
            jump p27eq0end, eq
            st   r1, r3, 4      // dataiorec[1]=r1
            move r1, 0x0000     // r1=0

            ld   r0, r3, 8      // r0=dataiorec[2] 
            add  r0, r0, 1      // r0++
            st   r0, r3, 8      // dataiorec[2]=r0

p27eq0end:

            reg_rd 0x3ff48424, 16, 17   // read IO register into r0
            st   r0, r3, 0      // dataiorec[0]=r0
            rsh  r0, r0, 12     // access pin 12
            and  r0, r0, 0x0001

            jump p12eq0, eq
            add  r2, r2, 0x0001
            jump p12eq0end
p12eq0:
            add  r2, r2, 0x0000 // update the zero flag
            jump p12eq0end, eq
            st   r2, r3, 12      // dataiorec[3]=r2
            move r2, 0x0000     // r2=0

            ld   r0, r3, 16     // r0=dataiorec[4] 
            add  r0, r0, 1      // r0++
            st   r0, r3, 16     // dataiorec[4]=r0

p12eq0end:
            jump mainloop
            halt
"""
binary = src_to_binary(source)
fout = open("measure_pwm.ulp", "wb")
fout.write(binary)
fout.close()

.0000 dataiorec
0001 timepin27
0002 countpin27
0003 timepin12
0004 countpin12
0005 entry
0008 mainloop
0015 p27eq0
0022 p27eq0end
0029 p12eq0
0036 p12eq0end


In [82]:
# Now we can load this assembled code into the ULP and set it running
# Don't forget to initialize the pins
%ls

Listing directory '/'.
     5039    BNO055_class.py
     4381    BNO055_funcs.py
     1123    UDPblackviewphone.py
      139    boot.py
      209    config.txt
             esp32_ulp/
      164    measure_pwm.ulp


In [67]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [83]:
import machine, time, esp32

p27 = machine.Pin(27, machine.Pin.IN)
p12 = machine.Pin(12, machine.Pin.IN)
print(p27.value(), p12.value())

ulp = esp32.ULP()
ulp.load_binary(0, open("measure_pwm.ulp", "rb").read())
ulp.set_wakeup_period(0, 1000000)
ulp.run(5*4)
def readulpdata(r):
    return machine.mem32[0x50000000+r]&0xFFFF



0 0


In [84]:
w = machine.PWM(machine.Pin(5))
w.freq(50)
w.duty(51)

import time
for i in range(20):
    print(readulpdata(4), readulpdata(8), readulpdata(12), readulpdata(16))
    if i == 5:
        w.duty(102)
    time.sleep(1)


111 162 1 1
112 212 1 1
112 262 1 1
112 312 1 1
112 362 1 1
112 412 1 1
224 462 1 1
223 512 1 1
224 560 1 1
748 608 1 1
.1110 658 224 50
1116 708 224 100
1119 759 224 150
1138 809 224 200
1090 861 224 250
1094 911 224 300
1107 961 224 350
1129 1013 224 400
1132 1064 224 450
1120 1114 224 500
.

In [ ]:
# Remainder of notebook is scratch space

In [9]:
import machine, time
p = machine.Pin(2, machine.Pin.OUT)
p.value(1)
time.sleep(1)
p.value(0)

p26 = machine.Pin(26, machine.Pin.IN, machine.Pin.PULL_DOWN)
p32 = machine.Pin(32, machine.Pin.IN)
p27 = machine.Pin(27, machine.Pin.IN, machine.Pin.PULL_DOWN)
p12 = machine.Pin(12, machine.Pin.IN)
print(p26.value(), p32.value(), p27.value(), p12.value())

0 0 1 0


In [10]:
print(p26.value(), p32.value(), p27.value(), p12.value())

0 0 1 0


In [18]:
#define READ_RTC_REG(rtc_reg, low_bit, bit_width) \
#    REG_RD (((rtc_reg) - DR_REG_RTCCNTL_BASE) / 4), ((low_bit) + (bit_width) - 1), (low_bit)
#	READ_RTC_FIELD (RTC_GPIO_IN_REG, RTC_GPIO_IN_NEXT)	// r0 = lower 16 RTC IOs
#DR_REG_RTCIO_BASE                       = 0x3ff48400
#define RTC_GPIO_IN_REG (DR_REG_RTCIO_BASE + 0x24)
#/* RTC_GPIO_IN_NEXT : RO ;bitpos:[31:10] ;default:  ; */
#/*description: RTC GPIO input data*/
#define RTC_GPIO_IN_NEXT 0x003FFFFF
#define RTC_GPIO_IN_NEXT_M ((RTC_GPIO_IN_NEXT_V) << (RTC_GPIO_IN_NEXT_S))
#define RTC_GPIO_IN_NEXT_V 0x3FFFFF
#define RTC_GPIO_IN_NEXT_S 10

#    GPIO 26 aka RTC 7
#    GPIO 33 aka RTC 8
#    GPIO 32 aka RTC 9
ulp.set_wakeup_period(0, -1)

In [19]:
from esp32_ulp.__main__ import src_to_binary
from esp32 import ULP

source = """\
data:       .long 0x55667788
data2:      .long 0x11223354
entry:      move r3, data    # load address of data into r3
            ld r2, r3, 0     # must be multiple of 4
            add r2, r2, 1
            move r3, data
            st r2, r3, 0

            // DR_REG_RTCIO_BASE                       = 0x3ff48400
            // Magic code for the 16-bit GPIO input register can be found here: https://github.com/espressif/esp-idf/blob/494a124d961c4c755685b22fe986c31826d6f503/components/soc/esp32s3/include/soc/rtc_io_reg.h#L94
            // list of pins here https://github.com/espressif/esp-idf/issues/4771
            move r0, 0x0000
            reg_rd 0x3ff48424, 16, 17
            move r3, data2
            st r0, r3, 0
            halt
"""
ulp = ULP()
binary = src_to_binary(source)
ulp.load_binary(0, binary)

from machine import mem32
print(hex(mem32[0x50000000]))
print(hex(mem32[0x50000004]))
ulp.run(2*4)
print(hex(mem32[0x50000000]))
print(hex(mem32[0x50000004]))


0000 data
0001 data2
0002 entry
0x55667788
0x11223354
0xc37789
0x1436500


In [77]:
from machine import mem32

print(hex(mem32[0x50000000]))
print(hex(mem32[0x50000004]))
ulp.run(2*4)
print(hex(mem32[0x50000000]))
print(hex(mem32[0x50000004]))


0x0
0x0
0x0
0x0


In [34]:
from esp32_ulp.__main__ import src_to_binary
from esp32 import ULP
ulp.set_wakeup_period(0, -1)

source = """\
dataiorec:  .long 0x00000000
timepin27:  .long 0x00000000
countpin27: .long 0x00000000
timepin12:  .long 0x00000000
countpin12: .long 0x00000000
entry:      move r3, dataiorec
            ld   r2, r3, 0      // r2=dataiorec[0] 
            add  r2, r2, 1      // r2++
            st   r2, r3, 0      // dataiorec[0]=r2

            move r1, 0x0000
loop1:
            reg_rd 0x3ff48424, 16, 17   // read IO register
            st  r0, r3, 4       // datacount[4]=r0
            rsh r0, r0, 14      // 
            and r0, r0, 0x0001  // accesses pin27

            jump p27eq0, eq
            add r1, r1, 0x0001
            jump p27eq0end
p27eq0:
            add r1, r1, 0x0000
            jump p27eq0end, eq
            st  r1, r3, 8
            move r1, 0x0000

            ld   r0, r3, 0      // r2=datacount[0] 
            add  r0, r0, 1      // r2++
            st   r0, r0, 0      // datacount[0]=r2

p27eq0end:
            jump loop1
            halt
"""

ulp = ULP()
binary = src_to_binary(source)
ulp.load_binary(0, binary)

from machine import mem32
print(hex(mem32[0x50000000]&0xFFFF), hex(mem32[0x50000004]&0xFFFF), hex(mem32[0x50000008]&0xFFFF))
ulp.run(4*4)
print(hex(mem32[0x50000000]&0xFFFF), hex(mem32[0x50000004]&0xFFFF), hex(mem32[0x50000008]&0xFFFF))


0000 datacount
0001 dataiorec
0002 datapin27
0003 entry
0008 loop1
0015 p27eq0
0022 p27eq0end
0x0 0x0 0x0
0x1 0x6500 0x0


In [58]:
from esp32_ulp.__main__ import src_to_binary
from esp32 import ULP
#ulp.set_wakeup_period(0, -1)
ulp.set_wakeup_period(0, 1000000)  # restart every 1 second (if halted)

source = """\
dataiorec:  .long 0x00000000
timepin27:  .long 0x00000000
countpin27: .long 0x00000000
timepin12:  .long 0x00000000
countpin12: .long 0x00000000
entry:
            move r3, dataiorec
            move r1, 0x0000     // pin27 counter
            move r2, 0x0000     // pin12 counter
mainloop:
            reg_rd 0x3ff48424, 16, 17   // read IO register into r0
            st   r0, r3, 0      // dataiorec[0]=r0
            rsh  r0, r0, 14     // access pin 27
            and  r0, r0, 0x0001

            jump p27eq0, eq
            add  r1, r1, 0x0001
            jump p27eq0end
p27eq0:
            add  r1, r1, 0x0000 // update the zero flag
            jump p27eq0end, eq
            st   r1, r3, 4      // dataiorec[1]=r1
            move r1, 0x0000     // r1=0

            ld   r0, r3, 8      // r0=dataiorec[2] 
            add  r0, r0, 1      // r0++
            st   r0, r3, 8      // dataiorec[2]=r0

p27eq0end:

            reg_rd 0x3ff48424, 16, 17   // read IO register into r0
            st   r0, r3, 0      // dataiorec[0]=r0
            rsh  r0, r0, 12     // access pin 12
            and  r0, r0, 0x0001

            jump p12eq0, eq
            add  r2, r2, 0x0001
            jump p12eq0end
p12eq0:
            add  r2, r2, 0x0000 // update the zero flag
            jump p12eq0end, eq
            st   r2, r3, 12      // dataiorec[3]=r2
            move r2, 0x0000     // r2=0

            ld   r0, r3, 16     // r0=dataiorec[4] 
            add  r0, r0, 1      // r0++
            st   r0, r3, 16     // dataiorec[4]=r0

p12eq0end:
            jump mainloop
            halt
"""

ulp = ULP()
binary = src_to_binary(source)
ulp.load_binary(0, binary)

from machine import mem32
print(hex(mem32[0x50000000]&0xFFFF), hex(mem32[0x50000004]&0xFFFF), hex(mem32[0x50000008]&0xFFFF))
ulp.run(5*4)
print(hex(mem32[0x50000000]&0xFFFF), hex(mem32[0x50000004]&0xFFFF), hex(mem32[0x50000008]&0xFFFF))


.0000 dataiorec
0001 timepin27
0002 countpin27
0003 timepin12
0004 countpin12
0005 entry
0008 mainloop
0015 p27eq0
0022 p27eq0end
0029 p12eq0
0036 p12eq0end
0x2500 0x0 0x0
0x2500 0x0 0x0


In [60]:
w = machine.PWM(machine.Pin(5))
w.freq(50)
w.duty(51)


In [61]:
w = machine.PWM(machine.Pin(5))
w.freq(50)
w.duty(51)

import time
for i in range(20):
    print(mem32[0x50000000]&0xFFFF, mem32[0x50000004]&0xFFFF, mem32[0x50000008]&0xFFFF, mem32[0x5000000C]&0xFFFF, mem32[0x50000010]&0xFFFF)
    if i == 5:
        w.duty(102)
    time.sleep(1)

9472 223 253 112 577
9472 223 253 112 627
9472 223 253 112 677
9472 223 253 112 727
9472 223 253 112 777
9472 223 253 112 825
9472 224 281 112 825
9472 224 331 112 825
9472 224 381 112 825
9472 224 431 112 825
.9472 224 481 112 825
9472 160 525 112 825
9472 160 525 224 834
9472 160 525 223 884
9472 160 525 223 934
9472 160 525 223 984
9472 160 525 224 1034
9472 160 525 224 1084
9472 160 525 224 1134
9472 160 525 224 1184
.

In [51]:
print(hex(13568-9472))

0x1000


In [ ]:
https://github.com/espressif/esp-idf/issues/4771

In [6]:
from esp32_ulp.__main__ import src_to_binary
from esp32 import ULP

source = """\
data:       .long 0x39000102
data2:      .long 0x11223344
entry:      rsh r3, r3, 16
            move r3, data    # load address of data into r3
            ld r2, r3, 2     # load data contents ([r3+0]) into r2
rep:        add r2, r2, 8    # increment r2
            st r2, r3, 2     # store r2 contents into data ([r3+0])
            #halt             # halt ULP co-prozessor (until it gets waked up again)
            #jump rep
kkk:        halt
"""
ulp = ULP()
binary = src_to_binary(source)
ulp.load_binary(0, binary)


0000 data
0001 data2
0002 entry
0005 rep
0007 kkk


In [7]:
ulp.set_wakeup_period(0, -1)

In [8]:
from machine import mem32
print(hex(mem32[0x50000000]))
print(hex(mem32[0x50000004]))

0xc3011a
0x11223344


In [ ]:
ulp.run(2*4)
